In [0]:
!pip3 install gpt-2-simple

In [0]:
import os
import time
import requests
import json
import gpt_2_simple as gpt2
import tensorflow as tf
import random
from datetime import datetime
from google.colab import drive
from google.colab import files
import numpy as np
import string
from functools import wraps
from time import time
#Mounts google drive
drive.mount('/content/drive')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
CHECKPOINT_DIR = "/content/drive/My Drive/GPT-2/checkpoints/"
SAMPLES_DIR = "/content/drive/My Drive/GPT-2/generated_outputs/"
DEBUG_MODE = False
MODEL_NAME = "124M" if DEBUG_MODE else "1558M"

In [0]:
!ls  "/content/drive/My Drive/GPT-2/checkpoints"
# !mkdir "checkpoint/run1/"

'Copy of checkpoint'
'Copy of encoder.json'
'Copy of events.out.tfevents.1579217689.2718339f40d8'
'Copy of hparams.json'
'Copy of model-1000.data-00000-of-00001'
'Copy of model-1000.index'
'Copy of model-1000.meta'
'Copy of vocab.bpe'
 infinite-infinite-jest.ipynb
 run1
'testing_text_generation_schemes1(1).ipynb'
 _testing_text_generation_schemes.ipynb
'Untitled folder'


In [0]:
config= {
    "length" : None,
    "return_as_list" : True,
    "include_prefix": True,
    "checkpoint_dir": CHECKPOINT_DIR,
    # "temperature" : 1.,
    # "top_k": 40,
    # "top_p": 0.9

}

In [0]:
tf.reset_default_graph()

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, 
               checkpoint_dir = CHECKPOINT_DIR)

Loading checkpoint /content/drive/My Drive/GPT-2/checkpoints/run1/model-1000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/GPT-2/checkpoints/run1/model-1000


In [0]:
import random

OUTPUT_FILEPATH = SAMPLES_DIR + "generated_output_" + MODEL_NAME + ".json"

def timing(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('Elapsed time: {}'.format((end-start)*1000))
        return result
    return wrapper


def remove_bad_patterns(text):
    data = text

    for ch in list(string.ascii_lowercase) + [","]:
        for n_spaces in range(0,5):
                pattern_match = ch + " " * n_spaces + "\n"
                data = data.replace(pattern_match, ch + " ")
    
    multiple_newlines = ["\n"*i for i in range(10,3,-1)] + ["\n \n"]    
    for k in multiple_newlines:
        data = data.replace(k,"\n")

    return data


def make_init_prompt(OUTPUT_FILEPATH=OUTPUT_FILEPATH):
    
    _output_dict = dict()
    _output_dict["id"] = 1
    _output_dict["prompt"] = config["prefix"]
    _output_dict["body"] = "I am seated in an office, surrounded by heads and bodies. \nMy posture is consciously congruent to the shape of my hard chair."
    _output_dict["params"] = {
        "temperature": None,
        "top_k": None
    }
    with open(OUTPUT_FILEPATH, mode='w') as f:
        json.dump([_output_dict], f, ensure_ascii=False, indent=4)
    return


def get_last_sequence_id():
    with open(OUTPUT_FILEPATH, mode='r') as f:
        print(OUTPUT_FILEPATH)
        f = json.load(f)
        return len(f)


def get_hiperparameters():
    temperature = np.random.choice([0.9,0.95,1])
    top_k = np.random.choice([30,40, 50, 60,80])
    return temperature, top_k


def set_prefix(chunk):

    #Preventing infinite dialogues, etc
    # if random.random() < 0.1:
    #     return None
    # else:
    with open(OUTPUT_FILEPATH, mode='r') as f:
        f = json.load(f)

    prompt =  f[-1]["body"]

    if len(f) == 1:
        return prompt
    else:
        tokens = min((len(prompt) // chunk), 1023)
        prompt = prompt[-tokens:]
        return prompt

def set_length():
    if config["prefix"]:
            length = 1023 - len(config["prefix"])
    else:
        length = 0
    return length

@timing
def generate_long_output(num_seq_to_gen=5,
                         config=config,
                         param_combination={}):

    if DEBUG_MODE or not os.path.isfile(OUTPUT_FILEPATH):
        make_init_prompt(OUTPUT_FILEPATH)

    current_id = get_last_sequence_id() + 1

    for curr_id in range(current_id, current_id + num_seq_to_gen):
        print("="*50)
        print(f"Sequence {curr_id-1}/{num_seq_to_gen}")

        chunk_of_last_seq_to_use_for_prompt = random.randint(5, 12)

        config["prefix"] = set_prefix(chunk_of_last_seq_to_use_for_prompt)
        config["length"] = set_length()

        temperature, top_k = get_hiperparameters()
        
        generated_batch = []

        if DEBUG_MODE:
            generated_batch = str(
                config["prefix"] + ((str(curr_id)+" ") * 100))
        else:
            print(f"Temperature: {temperature}, Top_k: {top_k}")
            print("Generating the sequence...")
            generated_batch = gpt2.generate(
                sess, **config, temperature=temperature, top_k=top_k)

        # Exclude prefix from the generated sequence
        if DEBUG_MODE:
            generated_batch = generated_batch[len(config["prefix"]):]
        elif config["prefix"]:
            generated_batch = generated_batch[0][len(config["prefix"]):]
        else:
            generated_batch = generated_batch[0]

        generated_batch = remove_bad_patterns(generated_batch)
        print("len(generated_batch): ", len(generated_batch))
        print("generated batch: ",print(generated_batch))

        _output_dict = {}
        _output_dict["id"] = curr_id
        _output_dict["prompt"] = config["prefix"]
        _output_dict["body"] = generated_batch
        _output_dict["params"] = {
            "temperature": float(temperature),
            "top_k": int(top_k),
            "chunk_size": chunk_of_last_seq_to_use_for_prompt,
            "length": config["length"]
        }

        with open(OUTPUT_FILEPATH, "r") as f:
            outputs = json.load(f)
            outputs.append(_output_dict)

        with open(OUTPUT_FILEPATH, "w") as f:
            json.dump(outputs, f, ensure_ascii=False, indent=4)




In [0]:
generate_long_output(num_seq_to_gen=10000)